# Merging and Joining DataFrames

1. **`pd.concat()`**:
   - Purpose: Concatenates DataFrames along a specified axis (rows or columns).
   - Usage: `pd.concat(objs, axis=0, join='outer', ignore_index=False)`
   - Parameters:
     - `objs`: Sequence of DataFrames to concatenate.
     - `axis`: Concatenate along rows (`axis=0`) or columns (`axis=1`).
     - `join`: Determines handling of columns not present in all DataFrames.
     - `ignore_index`: Create a new integer index if True.

2. **`pd.merge()`**:
   - Purpose: Merges DataFrames based on a common column or index.
   - Usage: `pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None)`
   - Parameters:
     - `left` and `right`: DataFrames to merge.
     - `how`: Type of merge - 'inner', 'outer', 'left', or 'right'.
     - `on`, `left_on`, `right_on`: Columns or indexes on which to merge.

3. **`pd.join()`**:
   - Purpose: Joins DataFrames based on their indexes.
   - Usage: `df.join(other, on=None, how='left')`
   - Parameters:
     - `other`: DataFrame to join with.
     - `on`: Column name or index level on which to join.
     - `how`: Type of join - 'left', 'right', or 'inner'.

In [ ]:
import pandas as pd
import numpy as np

#read the data
week1 = pd.read_csv("data/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("data/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("data/Restaurant - Customers.csv")
foods = pd.read_csv("data/Restaurant - Foods.csv")
satisfaction = pd.read_csv("data/Restaurant - Week 1 Satisfaction.csv")



/scratch/local/u1154915/715933/ipykernel_1160630/2891694628.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bigmac = pd.read_csv("data/bigmac.csv", parse_dates = ["Date"])


In [2]:
# take a look at the data
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [3]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [4]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [5]:
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


In [27]:
satisfaction.head()

,Satisfaction Rating
0,2
1,7
2,3
3,7
4,10


**`pd.concat()`**

In [11]:
# Concatenate week1 and week2 DataFrames into a single DataFrame called 'sales', with keys 'A' and 'B' to distinguish the two weeks
sales = pd.concat([week1, week2], keys=["A", "B"]) 
sales.head() 

Customer ID  Food ID
A 0          537        9
  1           97        4
  2          658        1
  3          202        2
  4          155        9

In [9]:
sales.loc[("A", 4), "Customer ID"] # Access the 'Customer ID' for the entry in week 'A' with index 4

np.int64(155)

**`df.merge()`**

In [13]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [14]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [ ]:
# Perform an inner merge between week1 and week2 DataFrames on 'Customer ID' and 'Food ID' columns to find common entries
res_week = week1.merge(week2, how="inner", on=["Customer ID", "Food ID"]) 
# An inner merge is used when you only want to analyze complete records where all necessary information exists across all source tables.
res_week.head(10)

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,922,1
6,21,4
7,578,5
8,578,5


In [ ]:
print('week1 shape:', week1.shape)
print('week2 shape:', week2.shape)  
print('res_week shape:', res_week.shape) #There are 9 common entries in both weeks with a similar customer ID and food ID

week1 shape: (250, 2)
week2 shape: (250, 2)
res_week shape: (9, 2)


In [ ]:
# Perform an outer merge between week1 and week2 DataFrames on 'Customer ID' column to include all entries from both weeks
res_week = week1.merge(week2, how="outer", on=["Customer ID"], suffixes = [" - Week 1", " - Week 2"], indicator = True)
# An outer merge is used when you want to retain all records from both source tables, regardless of whether there is a match. NaN values will be filled in for missing data.
res_week.head(10)

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
5,21,4.0,4.0,both
6,21,4.0,4.0,both
7,24,NaN,8.0,right_only
8,26,9.0,NaN,left_only
9,27,NaN,4.0,right_only


In [18]:
res_week["_merge"].value_counts() # Count the occurrences of each merge indicator to see how many entries are from week1 only, week2 only, or both weeks

_merge
right_only    197
left_only     195
both           62
Name: count, dtype: int64

In [19]:
mask = res_week["_merge"].isin(["left_only", "right_only"]) # Create a mask to filter entries that are unique to either week1 or week2
res_week[mask]

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [21]:
res_week = week1.merge(foods, how = "left", on = "Food ID", sort = True) # Merge week1 DataFrame with foods DataFrame to add food details to each sale record

res_week.head(10)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
5,77,1,Sushi,3.99
6,100,1,Sushi,3.99
7,953,1,Sushi,3.99
8,504,1,Sushi,3.99
9,323,1,Sushi,3.99


In [ ]:
Sushi = res_week[res_week["Food ID"]==1] # Filter the merged DataFrame to include only entries where 'Food ID' is 1 (Sushi)
print('Total Sushi sold in Week 1:', len(Sushi)) # Print the total number of Sushi sold in Week 1
Sushi.head()

Total Sushi sold in Week 1: 24


,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99


In [ ]:
# Merge week1 and week2 DataFrames using a left join on their indices to retain all entries from week1
res_week = week1.merge(week2, how = "left", left_index = True, 
                       right_index = True, suffixes = [" - Week 1", " - Week 2"])

res_week.head(10)

,Customer ID - Week 1,Food ID - Week 1,Customer ID - Week 2,Food ID - Week 2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
5,213,8,310,5
6,600,1,761,2
7,503,5,443,5
8,71,3,729,9
9,174,3,741,8


In [29]:
satisfaction.head()

,Satisfaction Rating
0,2
1,7
2,3
3,7
4,10


In [28]:
week1.join(satisfaction).head() # Join week1 DataFrame with satisfaction DataFrame using their indices to add satisfaction scores to each sale record

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


In [31]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [34]:
week2 = week2.sort_values(by="Customer ID") # Sort week2 DataFrame by 'Customer ID' to prepare for merging
week2.head(10)

,Customer ID,Food ID
16,8,6
24,13,2
30,21,4
128,24,8
197,27,4
193,30,4
149,35,8
191,39,10
243,45,8
134,46,6


In [30]:
# Merge week2 DataFrame with customers DataFrame to add customer details to each sale record
res_week = week2.merge(customers, how = "left", 
                       left_on = "Customer ID", 
                       right_on = "ID", sort = True)
res_week.head()

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,8,Frances,Adams,Female,Dabshots,Developer III
1,13,2,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,21,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,27,Jessica,Bennett,Female,Twitternation,Account Executive


Next Chapter [Working with Datetime](8.WorkingDateTimes.ipynb)